<a href="https://colab.research.google.com/github/ethanduncan65/Stock-Market-Backtest/blob/main/Stock_Market_Backtest_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Market Backtest Tool to Analyze Weekly Strategies

Instructions: 

1. Run the first cell below to install the yfinance package
1.   Enter ticker symbols for all of the stocks you want to analyze (ex: "MSFT, AAPL, FB")
2.   Select the date range you desire and then that cell
2. Run the third cell below to see a VIX chart for reference 
3. Download the analysis as a .csv file by entering a file name and running the last cell 


In [ ]:
pip install yfinance

In [ ]:
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format # forces 2 decimal places for floats in df's 
from pandas_datareader import data
from datetime import datetime
from datetime import timezone
import yfinance as yf
import matplotlib.pyplot as plt



# Gather user input and convert to list 
#@title Enter Stocks and Select Date Range 
Tickers = "MSFT, NFLX, AAPL, JPM" #@param {type:"string"}
Tickers_clean = Tickers.replace(" ", "")
Tickers_clean = Tickers_clean.upper()
tickers_list = list(Tickers_clean.split(","))

Start = "2014-10-15" #@param {type:"date"}
End = "2021-08-31" #@param {type:"date"}

pd_startDate = pd.Timestamp(Start)
pd_endDate = pd.Timestamp(End)

# http://www.market-holidays.com/
# any holidays that fall on a Monday or Friday are automatically excluded
holidays = ['2021-11-25', 
            '2020-11-26', '2020-01-01',
            '2019-12-25', '2019-11-28', '2019-07-04', '2019-01-01', 
            '2018-12-25', '2018-11-22', '2018-07-04', 
            '2017-07-04', '2017-11-23',
            '2016-11-24',
            '2015-01-01', '2015-11-26',
            '2014-01-01', '2014-11-27', '2014-12-25',
            '2013-01-01', '2013-07-04', '2013-11-28', '2013-12-25',
            '2012-07-04', '2012-11-22', '2012-12-25',
            '2011-11-24',
            '2010-11-25',
            ] ##### NEED TO ADD REST OF YEARS


def checkMonday(k, df):
  i_date = df.iloc[k]['Date']
  # if the previous day is a holiday rather than sunday, return false
  is_holiday = checkPostHoliday(i_date)
  if is_holiday:
    return(False)
  else:
    i_dateNum = i_date.replace(tzinfo=timezone.utc).timestamp()
    iminus1_date = df.iloc[k-1]['Date']
    iminus1_dateNum = iminus1_date.replace(tzinfo=timezone.utc).timestamp()
    delta = (i_dateNum - iminus1_dateNum)
    if delta>86400:
      return(True)
    else: 
      return(False)


def checkFriday(k, df):
  i_date = df.iloc[k]['Date']
  # if the next day is a holiday rather than saturday, return false
  is_holiday = checkPreHoliday(i_date)
  if is_holiday:
    return(False)
  else:
    i_dateNum = i_date.replace(tzinfo=timezone.utc).timestamp()
    iplus1_date = df.iloc[k+1]['Date']
    iplus1_dateNum = iplus1_date.replace(tzinfo=timezone.utc).timestamp()
    delta = (iplus1_dateNum - i_dateNum)
    if delta>86400:
      return(True)
    else: 
      return(False)


def checkPostHoliday(date):
  date_num = date.replace(tzinfo=timezone.utc).timestamp()
  date_numPlus1Day = date_num - 86400
  date_numPlus1Day_dtObjStr = str(datetime.fromtimestamp(date_numPlus1Day))
  date_to_compare = date_numPlus1Day_dtObjStr[:10]
  for x in range(0, len(holidays)):
    if (date_to_compare==holidays[x]):
      return(True)
  return(False)


def checkPreHoliday(date):
  date_num = date.replace(tzinfo=timezone.utc).timestamp()
  date_numPlus1Day = date_num + 86400
  date_numPlus1Day_dtObjStr = str(datetime.fromtimestamp(date_numPlus1Day))
  date_to_compare = date_numPlus1Day_dtObjStr[:10]
  for x in range(0, len(holidays)):
    if (date_to_compare==holidays[x]):
      return(True)
  return(False)

df_final = pd.DataFrame(columns = ['Ticker', 'Stock', '+50%', '+40%', '+30%', '+25%', 
                                   '+20%', '+15%', '+10%', '+9%', '+8%', '+7%', 
                                   '+6%', '+5%', '+4%', '+3%', '+2%', '+1%', 
                                   '0%', '-1%', '-2%', '-3%', '-4%', '-5%', 
                                   '-6%', '-7%', '-8%', '-9%', '-10%', '-15%', 
                                   '-20%', '-25%', '-30%', '-40%', '-50%'], 
                          index = range(0, len(tickers_list))) 

# loop through each stock from user input
for q in range(0, len(tickers_list)):
  q_ticker = yf.Ticker(tickers_list[q])
  q_name = q_ticker.info.get('shortName')
  df_yf = q_ticker.history(period="max")
  df_yfClean = df_yf.reset_index()
  df_yfFinal = df_yfClean[df_yfClean['Date'] >= pd_startDate]
  df = df_yfFinal[df_yfFinal['Date'] <= pd_endDate]
  df2 = df.reset_index()
  # Create custom df to be returned to user at end
  dfOutput = pd.DataFrame(columns = ['Start Date', 'End Date', 'Open Price', 
                                     'Close Price', 'Change Amt.', 'Change %'], 
                          index = range(0, 2500)) 
  # iterator variable to move through the rows of dfOutput
  weekNum = 0
  # loop through each day to determine weekly strcuture 
  for i in range(0, len(df2)):
    # is this the first row?
    if i==0: 
      # tfx first set of data & continue
      dfOutput.at[weekNum, 'Start Date'] = df2.iloc[i]['Date']
      dfOutput.at[weekNum, 'Open Price'] = df2.iloc[i]['Open']
      continue
    if i==(len(df2)-1):
      friday_date = df2.iloc[i]['Date']
      friday_closePrice = df2.iloc[i]['Close']
      # export the data to dfOutput
      dfOutput.at[weekNum, 'End Date'] = friday_date
      dfOutput.at[weekNum, 'Close Price'] = friday_closePrice
      continue 
    # is it a monday/starting day of the week?
    is_i_monday = checkMonday(i, df2)
    if is_i_monday:
      # tfx monday's data
      # import data from df2
      monday_date = df2.iloc[i]['Date']
      monday_openPrice = df2.iloc[i]['Open']
      # export the data to dfOutput
      dfOutput.at[weekNum, 'Start Date'] = monday_date
      dfOutput.at[weekNum, 'Open Price'] = monday_openPrice
    else:
      # is it a friday/ending day of the week?
      is_i_friday = checkFriday(i, df2)
      if is_i_friday:
        # tfx friday's data
        # import data from df2
        friday_date = df2.iloc[i]['Date']
        friday_closePrice = df2.iloc[i]['Close']
        # export the data to dfOutput
        dfOutput.at[weekNum, 'End Date'] = friday_date
        dfOutput.at[weekNum, 'Close Price'] = friday_closePrice
        # after tfx'ing friday's data, iterate weekNum to move to next row on dfOutput
        weekNum+=1
      else:
        # skip this row/day (tuesday/wednesday/thursday)
        continue
  # loop through each week and calc. the change amount and %
  for z in range(0, len(dfOutput)):
    # record n/a for first week
    if z==0:
      dfOutput.at[z, 'Change Amt.'] = 0
      dfOutput.at[z, 'Change %'] = 0
      continue
    changeAmt = dfOutput.iloc[z]['Close Price'] - dfOutput.iloc[z-1]['Close Price']
    changePct = ((changeAmt)/dfOutput.iloc[z-1]['Close Price'])*100
    # add calculations to dfOutput
    dfOutput.at[z, 'Change Amt.'] = changeAmt
    dfOutput.at[z, 'Change %'] = changePct

  # remove any blank rows before returning the final df
  dfOutput1 = dfOutput.dropna()
  dfOutput2 = dfOutput1.iloc[1: , :]
  q_dfOutput = dfOutput2.reset_index()
  
  # add rates to final dataframe
  df_final.at[q, 'Ticker'] = tickers_list[q]
  df_final.at[q, 'Stock'] = q_name
  df_final.at[q, '+50%'] = len(q_dfOutput[(q_dfOutput['Change %']>50)])/len(q_dfOutput)
  df_final.at[q, '+40%'] = len(q_dfOutput[(q_dfOutput['Change %']>40)])/len(q_dfOutput)
  df_final.at[q, '+30%'] = len(q_dfOutput[(q_dfOutput['Change %']>30)])/len(q_dfOutput)
  df_final.at[q, '+25%'] = len(q_dfOutput[(q_dfOutput['Change %']>25)])/len(q_dfOutput)
  df_final.at[q, '+20%'] = len(q_dfOutput[(q_dfOutput['Change %']>20)])/len(q_dfOutput)
  df_final.at[q, '+15%'] = len(q_dfOutput[(q_dfOutput['Change %']>15)])/len(q_dfOutput)
  df_final.at[q, '+10%'] = len(q_dfOutput[(q_dfOutput['Change %']>10)])/len(q_dfOutput)
  df_final.at[q, '+9%'] = len(q_dfOutput[(q_dfOutput['Change %']>9)])/len(q_dfOutput)
  df_final.at[q, '+8%'] = len(q_dfOutput[(q_dfOutput['Change %']>8)])/len(q_dfOutput)
  df_final.at[q, '+7%'] = len(q_dfOutput[(q_dfOutput['Change %']>7)])/len(q_dfOutput)
  df_final.at[q, '+6%'] = len(q_dfOutput[(q_dfOutput['Change %']>6)])/len(q_dfOutput)
  df_final.at[q, '+5%'] = len(q_dfOutput[(q_dfOutput['Change %']>5)])/len(q_dfOutput)
  df_final.at[q, '+4%'] = len(q_dfOutput[(q_dfOutput['Change %']>4)])/len(q_dfOutput)
  df_final.at[q, '+3%'] = len(q_dfOutput[(q_dfOutput['Change %']>3)])/len(q_dfOutput)
  df_final.at[q, '+2%'] = len(q_dfOutput[(q_dfOutput['Change %']>2)])/len(q_dfOutput)
  df_final.at[q, '+1%'] = len(q_dfOutput[(q_dfOutput['Change %']>1)])/len(q_dfOutput)
  df_final.at[q, '0%'] = len(q_dfOutput[(q_dfOutput['Change %']>0)])/len(q_dfOutput)
  df_final.at[q, '-1%'] = len(q_dfOutput[(q_dfOutput['Change %']>-1)])/len(q_dfOutput)
  df_final.at[q, '-2%'] = len(q_dfOutput[(q_dfOutput['Change %']>-2)])/len(q_dfOutput)
  df_final.at[q, '-3%'] = len(q_dfOutput[(q_dfOutput['Change %']>-3)])/len(q_dfOutput)
  df_final.at[q, '-4%'] = len(q_dfOutput[(q_dfOutput['Change %']>-4)])/len(q_dfOutput)
  df_final.at[q, '-5%'] = len(q_dfOutput[(q_dfOutput['Change %']>-5)])/len(q_dfOutput)
  df_final.at[q, '-6%'] = len(q_dfOutput[(q_dfOutput['Change %']>-6)])/len(q_dfOutput)
  df_final.at[q, '-7%'] = len(q_dfOutput[(q_dfOutput['Change %']>-7)])/len(q_dfOutput)
  df_final.at[q, '-8%'] = len(q_dfOutput[(q_dfOutput['Change %']>-8)])/len(q_dfOutput)
  df_final.at[q, '-9%'] = len(q_dfOutput[(q_dfOutput['Change %']>-9)])/len(q_dfOutput)
  df_final.at[q, '-10%'] = len(q_dfOutput[(q_dfOutput['Change %']>-10)])/len(q_dfOutput)
  df_final.at[q, '-15%'] = len(q_dfOutput[(q_dfOutput['Change %']>-15)])/len(q_dfOutput)
  df_final.at[q, '-20%'] = len(q_dfOutput[(q_dfOutput['Change %']>-20)])/len(q_dfOutput)
  df_final.at[q, '-25%'] = len(q_dfOutput[(q_dfOutput['Change %']>-25)])/len(q_dfOutput)
  df_final.at[q, '-30%'] = len(q_dfOutput[(q_dfOutput['Change %']>-30)])/len(q_dfOutput)
  df_final.at[q, '-40%'] = len(q_dfOutput[(q_dfOutput['Change %']>-40)])/len(q_dfOutput)
  df_final.at[q, '-50%'] = len(q_dfOutput[(q_dfOutput['Change %']>-50)])/len(q_dfOutput)

# output final dataframe
df_final

In [ ]:
#@title VIX Chart for Reference
# plot of VIX for reference
import plotly.express as px

vix_data = yf.Ticker('^VIX')
vix_history = vix_data.history(period="max")
vix_historyC = vix_history.reset_index()

vix_historyC1 = vix_historyC[vix_historyC['Date'] >= pd_startDate]
vix_df = vix_historyC1[vix_historyC1['Date'] <= pd_endDate]

#vix_dates = vix_df['Date'].tolist()
#vix_prices = vix_df['Close'].tolist()

fig = px.line(vix_df, x="Date", y="Close", title='VIX (CBOE Volatility Index) Daily Close Prices')
fig.show()

#fig = plt.figure()
#plt.plot(vix_dates, vix_prices)

In [ ]:
#@title Enter File Name and Download df as .csv 
file_name = "" #@param {type:"string"}
file_name_csv = file_name + '.csv'
# download the df as .csv file using input name
df_final.to_csv(file_name_csv)